In [1]:
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import plotly.express as px
from geopy.geocoders import Nominatim
import time
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact

# Define the API endpoint and parameters
url = "https://api.fda.gov/food/enforcement.json?search=report_date:[20200101+TO+20241201]&limit=1000"

# Send a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    recalls = data.get('results', [])
    recall_df = pd.DataFrame(recalls)

    clean_df = recall_df.rename(columns={
        "status": "Status", 
        "city": "City",
        "state": "State",
        "country":"Country",
        "classification":"Classification",
        "openfda":"Open FDA",
        "product_type":"Product Type",
        "event_id":"Event ID",
        "recalling_firm":"Recalling Firm",
        "address_1":"Address 1",
        "address_2":"Address 2",
        "postal_code":"Postal Code",
        "voluntary_mandated":"Voluntary Mandated",
        "initial_firm_notification":"Initial Firm Notification",
        "distribution_pattern":"Distribution Pattern",
        "recall_number":"Recall Number",
        "product_description":"Product Description",
        "product_quantity":"Product Quantity",
        "reason_for_recall":"Reason for Recall",
        "recall_initiation_date":"Recall Initiation Date",
        "center_classification_date":"Center Classification Date",
        "termination_date":"Termination Date",
        "report_date":"Report Date",
        "code_info":"Code Info",
        "more_code_info":"More Code Info",
    })

    # Convert DataFrame to dictionary format for MongoDB
    recall_records = clean_df.to_dict("records")

    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Or use your MongoDB URI
    db = client['fda_recall_data']  # Database name
    collection = db['recalls']  # Collection name

    # Insert modified data into MongoDB
    collection.insert_many(recall_records)

    # Example: Query data back from MongoDB
    recalls_from_db = list(collection.find())

    # Convert back to DataFrame if needed
    db_df = pd.DataFrame(recalls_from_db)
    
else:
    print(f"Failed to retrieve data: {response.status_code}")

In [7]:
recalls_from_db


[{'_id': ObjectId('67071919c447fc77ab6a1cd0'),
  'Status': 'Terminated',
  'City': 'Rancho Dominguez',
  'State': 'CA',
  'Country': 'United States',
  'classification': 'Class III',
  'openfda': {},
  'product_type': 'Food',
  'event_id': '85253',
  'recalling_firm': 'Organic By Nature, Inc.',
  'address_1': '2610 Homestead Pl',
  'address_2': 'N/A',
  'postal_code': '90220-5610',
  'voluntary_mandated': 'Voluntary: Firm initiated',
  'initial_firm_notification': 'Two or more of the following: Email, Fax, Letter, Press Release, Telephone, Visit',
  'distribution_pattern': 'nationwide, Canada and Netherlands',
  'recall_number': 'F-0904-2020',
  'product_description': 'Pure Planet Organic Parasite Cleanse;  Net Wt. 174g Glass Jar;  Finished Product Item # 52700    Manufactured and Distributed by:  Pure Planet,  Rancho Dominguez, CA',
  'product_quantity': 'xx',
  'reason_for_recall': 'Firm was notified by supplier that Organic Ground Flaxseed powder was under recall by manufacturer due